# HMDB database importer

In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import ast
import re

import bisect

In [3]:

def triangle_overlap_area(x_a, y_a, x_b, y_b, b):
    """
    Compute the overlap area between two isosceles triangles with:
        - same base length b
        - bases on y = 0
        - centers at x_a and x_b on the x-axis
        - heights y_a and y_b
    """
    def tri_height(x, xc, h, b):
        """Return the height of the triangle at position x (piecewise linear)."""
        half = b / 2
        left, right = xc - half, xc + half
        if x < left or x > right:
            return 0.0
        # Linear interpolation
        if x <= xc:
            return h * (x - left) / (xc - left)
        else:
            return h * (right - x) / (right - xc)

    # Compute global horizontal overlap range
    L = max(x_a - b/2, x_b - b/2)
    R = min(x_a + b/2, x_b + b/2)

    if R <= L:
        return 0.0  # No overlap

    # Integrate numerically (analytical solution is possible but long)
    # Fine grid for accuracy
    xs = np.linspace(L, R, 2001)
    h_min = np.minimum(
        [tri_height(x, x_a, y_a, b) for x in xs],
        [tri_height(x, x_b, y_b, b) for x in xs]
    )

    # Numerical integration using trapezoid rule
    area = np.trapz(h_min, xs)
    return area


# Range query function
def range_query_and(query, nmrdb):
    indexer = [(a, b) for a, b in zip(nmrdb['shift1(ppm)'].values, nmrdb.index.values)]
    result_ids = None
    for rng in query:
        range_i = rng['range']
        left_index = bisect.bisect_left(indexer,  (range_i[0], -1))
        right_index = bisect.bisect_right(indexer, (range_i[1], float('inf')))
        res = set(nmrdb.loc[[item[1] for item in indexer[left_index:(right_index + 1)]],['accession']]['accession'].values)
        if result_ids is None:
            result_ids = res
        else:
            result_ids = result_ids & res
    return list(result_ids)

def multiplet_match(test_ppm, test_heights, query_ppm, query_heights):
    #1.  Center both multiplets at center of mass
    cm_test = np.sum(test_ppm * test_heights) / np.sum(test_heights)
    test_ppm -= cm_test
    cm_query = np.sum(query_ppm * query_heights) / np.sum(query_heights)
    query_ppm -= cm_query
    #2. Compare area overlap when both multiplets has area 1.
    score =0
    base = 0.008 # TODO: We could change this. 
    for i in range(test_ppm.shape[0]):
        for j in range(query_ppm.shape[0]):
            score += triangle_overlap_area(test_ppm[i], test_heights[i], query_ppm[j], query_heights[j], base)
    assert (score >=0) & (score <= 1), "Score out of range"
    return score * 2 / base    
    

def multiple_query(query, nmrdb, metabolytes):
    res = range_query_and(query, nmrdb)
    filtered_df = metabolytes.loc[metabolytes['accession'].isin(res), ]
    result = []
    
    # Ensure correct query structure
    for query_i in query:
        if 'ppm' in query_i:
            query_i['ppm'] = np.array(query_i['ppm'])
            query_i['heights'] = np.array(query_i['heights'])
            query_i['heights'] /= np.sum(query_i['heights'])
        
    for row in filtered_df.iterrows():
        signals = nmrdb[nmrdb['accession'] == row[1]["accession"]]
        matches = 0
        for query_i in query:
            range_i = query_i['range']
            mult = query_i['mult']
            from_i = range_i[0]
            to_i = range_i[1]
            mul_i = mult
            for signal in signals.iterrows():
                if signal[1]['shift1(ppm)'] >= from_i and signal[1]['shift1(ppm)'] <= to_i and ((signal[1]['type'] == mul_i) or (mul_i == "*")):
                    matches += 1
                    # Check multiplet alignment
                    if 'ppm' in query_i:
                        matches += multiplet_match(signal[1]['ppm'], signal[1]['heights'], query_i['ppm'], query_i['heights'])
                    break
        result.append((row[1]["accession"], row[1]["name"], len(query) / signals.shape[0] + matches /  len(query)))
                    
    result = pd.DataFrame(result, columns=["accession", "name", "similarity"])
    result = result.sort_values("similarity", ascending=False)
    return result

def count_atoms(formula):
    """
    Counts the number of atoms of each element in a chemical formula.

    Args:
        formula: A string representing the chemical formula.

    Returns:
        A dictionary where keys are element symbols and values are the number of atoms of that element.
    """
    atoms = {}
    for match in re.findall(r"([A-Z][a-z]*)(\d*)", formula):
        element = match[0]
        count = int(match[1]) if match[1] else 1
        atoms[element] = atoms.get(element, 0) + count
    return atoms

# Apply the function to the 'chemical_formula' column
#atom_counts = df['chemical_formula'].apply(count_atoms)

# Expand the dictionary of atom counts into separate columns
#df = pd.concat([df, atom_counts.apply(pd.Series)], axis=1)
#print(df)


## Read compiled DB

In [4]:
nmrdb = pd.read_csv("inst/spectral1hnmr.csv")
df = pd.read_csv("inst/metabolites.csv")

# Convert ppm from string to np.array
nmrdb['ppm'] = nmrdb['ppm'].apply(lambda x: np.array(ast.literal_eval(x.replace("'",''))))

def h_converter(x):
    x = np.array(ast.literal_eval(x.replace("'",'')))
    return x / np.sum(x) 
    
nmrdb['heights'] = nmrdb['heights'].apply(lambda x: h_converter(x))

## Rank matches according to multiplet similarity

In [5]:
query = [ {'range': (3.87, 3.93), 'mult': 'dd', 'ppm': np.array([3.8892, 3.8930, 3.9097, 3.9134]), 'heights': np.array([0.25, 0.25, 0.25, 0.25])},]

result = multiple_query(query, nmrdb, df)
result.head(10) 

,accession,name,similarity
6,HMDB0000122,D-Glucose,2.126998
13,HMDB0000191,L-Aspartic acid,1.965755
39,HMDB0000884,Ribothymidine,1.952838
32,HMDB0000660,D-Fructose,1.942185
18,HMDB0000258,Sucrose,1.905719
35,HMDB0000742,Homocysteine,1.898036
58,HMDB0002006,"2,3-Diaminopropionic acid",1.894190
59,HMDB0002545,Galacturonic acid,1.865068
20,HMDB0000296,Uridine,1.828641
9,HMDB0000158,L-Tyrosine,1.786886


In [6]:
query = [{'range': (2.5, 2.6), 'mult': 'd'}, {'range': (2.6, 2.7), 'mult': 'd'}]

result = multiple_query(query, nmrdb, df)
result.head()

,accession,name,similarity
1,HMDB0000094,Citric acid,2.000000
3,HMDB0000402,2-Isopropylmalic acid,1.400000
4,HMDB0000736,Isobutyryl-L-carnitine,0.722222
7,HMDB0001257,Spermidine,0.666667
8,HMDB0001844,Methylsuccinic acid,0.500000


In [7]:
query = [{'range':(1.25, 1.35), 'mult': 'd'}, {'range': (4.05, 4.15), 'mult': 'q'}]

result = multiple_query(query, nmrdb, df)
result.head(6)    

,accession,name,similarity
1,HMDB0000190,L-Lactic acid,2.000000
0,HMDB0000030,Biotin,0.700000
2,HMDB0000701,Hexanoylglycine,0.400000
4,HMDB0002802,Cortisone,0.142857
3,HMDB0001348,SM(d18:1/18:0),0.133333
5,HMDB0014352,Azithromycin,0.062500


In [8]:
query = [{'range': (3.87, 3.93), 'mult': '*'},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
2,HMDB0000064,Creatine,1.500000
1,HMDB0000043,Betaine,1.500000
58,HMDB0002006,"2,3-Diaminopropionic acid",1.500000
55,HMDB0001991,7-Methylxanthine,1.500000
35,HMDB0000742,Homocysteine,1.333333
64,HMDB0011599,1-Methyladenine,1.333333


In [9]:
query = [{'range': (3.87, 3.93), 'mult': 'dd'},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
58,HMDB0002006,"2,3-Diaminopropionic acid",1.500000
35,HMDB0000742,Homocysteine,1.333333
9,HMDB0000158,L-Tyrosine,1.200000
26,HMDB0000479,3-Methylhistidine,1.200000
12,HMDB0000181,L-Dopa,1.166667
13,HMDB0000191,L-Aspartic acid,1.166667


In [10]:
nmrdb[nmrdb['accession']=='HMDB0000742']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
1055,HMDB0000742,m03,2.14,1,"['14.86', '14.70', '7.47']",[3],m,2.06,2.22,"[2.07, 2.09, 2.1, 2.12, 2.13, 2.14, 2.15, 2.15...","[1037.1, 1045.1, 1051.7, 1058.8, 1066.2, 1068....","[0.018460648148148146, 0.028732638888888884, 0...",[3]
1415,HMDB0000742,m02,2.65,2,"['10.73', '8.12', '6.73']",[2],ddd,2.59,2.72,"[2.6, 2.62, 2.63, 2.64, 2.64, 2.65, 2.66, 2.66...","[1300.1, 1307.2, 1313.7, 1320.7, 1321.7, 1324....","[0.029597332649397283, 0.036881251602975125, 0...",[2]
2231,HMDB0000742,m01,3.87,1,"['7.13', '5.62']",[4],dd,3.83,3.90,"[-0.01001467351430696, -1.4673514306728919e-05...","[1928.4, 1934.0, 1935.5, 1941.1]","[0.23097445474554795, 0.2749282998732742, 0.26...",[4]


In [11]:
# Looking for Leucine
query = [{'range': (0.94, 0.99), 'mult': 't', 'ppm': np.array([0.949542,0.96010, 0.970836]), 'heights': np.array([0.25,0.5,0.25])},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
9,HMDB0000452,L-alpha-Aminobutyric acid,2.241946
17,HMDB0000687,L-Leucine,2.154733
16,HMDB0000650,D-alpha-Aminobutyric acid,1.754231
27,HMDB0001987,2-Hydroxy-2-methylbutyric acid,1.712065
7,HMDB0000339,2-Methylbutyrylglycine,1.563549
24,HMDB0001388,alpha-Linolenic acid,1.392834


In [12]:
# Looking for Threonine
query = [{'range': (4.22, 4.28), 'mult': '*', 'ppm': np.array([4.2351,4.243,4.2461,4.254 ,4.2571,4.265 ,4.2681,4.276]), 'heights': np.array([1,1,3,3,3,3,1,1]) / np.sum([1,1,3,3,3,3,1,1])},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
1,HMDB0000167,L-Threonine,2.232421
3,HMDB0000244,Riboflavin,2.166881
8,HMDB0000565,Galactonic acid,1.909901
14,HMDB0000982,5-Methylcytidine,1.905070
16,HMDB0001563,1-Methylguanosine,1.892107
18,HMDB0002545,Galacturonic acid,1.888997


In [13]:
# Looking for Tyrosine
query = [{'range': (7.16, 7.6), 'mult': '*', 'ppm': [7.185000,7.189691,7.192445,7.200093,7.203968,7.208965], 'heights': [0.14,1,0.3,0.28,0.92,0.14]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
3,HMDB0000158,L-Tyrosine,2.297146
88,HMDB0004811,"2,4-Dichlorophenol",2.268032
92,HMDB0005794,Quercetin,2.252696
5,HMDB0000205,Phenylpyruvic acid,2.178898
84,HMDB0003312,Daidzein,2.154630
74,HMDB0002055,o-Cresol,2.142429


In [14]:
# Looking for Phenylalanine
query = [{'range': (7.2, 7.5), 'mult': '*', 'ppm': [7.414916,7.418995,7.421035,7.429193,7.431233,7.434292,7.440411,7.442960,7.4450005], 'heights': [0.16,0.8,0.2,0.53,1,0.2,0.13,0.33,0.2]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
22,HMDB0000684,L-Kynurenine,2.466096
70,HMDB0004812,"2,5-Furandicarboxylic acid",2.396479
54,HMDB0001942,Phenylpropanolamine,2.301726
5,HMDB0000228,Phenol,2.286237
24,HMDB0000715,Kynurenic acid,2.238970
3,HMDB0000205,Phenylpyruvic acid,2.234585


In [15]:
# Looking for Lactate
query = [{'range': (4, 4.2), 'mult': 'q', 'ppm': [4.095,4.1065,4.118,4.1295], 'heights': [0.33,1,1,0.33]},]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
21,HMDB0000190,L-Lactic acid,2.205696
12,HMDB0000125,Glutathione,1.832804
20,HMDB0000174,L-Fucose,1.748530
10,HMDB0000101,Deoxyadenosine,1.740319
85,HMDB0003072,Quinic acid,1.375292
81,HMDB0002721,1-Methylinosine,1.325114


In [16]:
nmrdb[nmrdb['accession'] == 'HMDB0000190']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
442,HMDB0000190,m01,1.32,4,['6.96'],['3'],d,1.30,1.33,"[1.31, 1.324]","['654.7', '661.7']","[0.49952454832090487, 0.5004754516790951]",['3']
2440,HMDB0000190,m02,4.10,2,"['6.93', '6.93']",['2'],q,4.08,4.13,"[-0.020341658481831004, -0.00634165848183077, ...","['2041.1', '2048.1', '2055.0', '2061.9']","[0.12670127683457275, 0.378981338571629, 0.373...",['2']


In [17]:
# Looking for Lactate
query = [{'range': (4, 4.2), 'mult': 'q', 'ppm': [4.095,4.1065,4.118,4.1295], 'heights': [0.33,1,1,0.33]},
         {'range': (1.25, 1.36), 'mult': 'd', 'ppm': [1.31, 1.324]	, 'heights': [0.49952454832090487, 0.5004754516790951]	}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
4,HMDB0000190,L-Lactic acid,2.852848
3,HMDB0000174,L-Fucose,1.822182
0,HMDB0000030,Biotin,0.824622
5,HMDB0000424,2-Hydroxydecanedioic acid,0.400000
6,HMDB0000701,Hexanoylglycine,0.400000
10,HMDB0001924,Atenolol,0.250000


In [18]:
nmrdb[nmrdb['accession'] == 'HMDB0000174']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
345,HMDB0000174,m09,1.21,8,['6.65'],['22'],d,1.20,1.22,"[1.22, 1.21]","['729.2', '722.5']","[0.513590033975085, 0.486409966024915]",['22']
366,HMDB0000174,m08,1.25,18,['6.48'],['11'],d,1.24,1.27,"[0.004998405103668224, -0.005001594896331785]","['753.2', '746.7']","[0.5001594896331738, 0.4998405103668262]",['11']
1824,HMDB0000174,m07,3.45,9,"['9.89', '7.97']",['4'],dd,3.42,3.48,"[3.47, 3.45, 3.45, 3.44]","['2078.0', '2070.1', '2068.2', '2060.1']","[0.2532330827067669, 0.2706766917293233, 0.265...",['4']
1968,HMDB0000174,m06,3.64,8,"['9.92', '3.49']",['3'],dd,3.62,3.67,"[3.66, 3.65, 3.64, 3.63]","['2191.4', '2187.9', '2181.5', '2178.0']","[0.23790849673202616, 0.2173202614379085, 0.28...",['3']
2124,HMDB0000174,m05,3.79,24,[''],"['5', '14', '16', '6']",m,3.73,3.83,"[3.82, 3.81, 3.8, 3.79, 3.78, 3.78, 3.77, 3.76...","['2292.6', '2286.1', '2279.6', '2273.2', '2267...","[0.046151282905698095, 0.19976674441852715, 0....","['5', '14', '16', '6']"
2220,HMDB0000174,m04,3.86,4,"['10.30', '3.38']",['15'],dd,3.84,3.89,"[3.87, 3.87, 3.86, 3.85]","['2322.5', '2319.1', '2312.2', '2308.8']","[0.21412964311726146, 0.2199563000728332, 0.24...",['15']
2502,HMDB0000174,m03,4.20,3,['6.61'],['17'],q,4.17,4.23,"[0.016282352941176234, 0.0062823529411764455, ...","['2527.9', '2521.3', '2514.7', '2508.1']","[0.11882352941176472, 0.3776470588235294, 0.38...",['17']
2661,HMDB0000174,m02,4.56,6,['7.91'],['2'],d,4.53,4.58,"[4.57, 4.55]","['2737.5', '2729.5']","[0.49450286806883365, 0.5054971319311663]",['2']
2741,HMDB0000174,m01,5.21,3,['3.90'],['13'],d,5.20,5.22,"[5.21, 5.21]","['3124.0', '3120.1']","[0.4947751022262608, 0.5052248977737392]",['13']


In [19]:
# Looking for Ibupeofen
query = [{'range': (7.07, 7.35), 'mult': '*', 'ppm': [7.122, 7.136, 7.241, 7.254], 'heights': [0.8,1,1,0.8]},]
     #   {'range': (0.88, 0.96), 'mult': 'd', 'ppm': [0.916, 0.927], 'heights': [1.0,1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
75,HMDB0004812,"2,5-Furandicarboxylic acid",2.000000
59,HMDB0001928,Hydrochlorothiazide,1.500000
65,HMDB0002085,Syringic acid,1.500000
32,HMDB0000764,Hydrocinnamic acid,1.440424
0,HMDB0000020,p-Hydroxyphenylacetic acid,1.333333
9,HMDB0000228,Phenol,1.333333


In [20]:
nmrdb[nmrdb['accession'] == 'HMDB0000667']

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
1478,HMDB0000667,m07,2.80,1,[''],['15'],m,2.76,2.85,"[2.79, 2.8, 2.81, 2.82]",[1676.79 1682.8 1688.81 1694.82],"[0.22075869336143308, 0.24130663856691253, 0.2...",[15]
1615,HMDB0000667,m06,3.08,1,['4.35'],['15'],d,3.03,3.15,"[3.07, 3.08, 3.1, 3.1]",[1845.07 1851.08 1863.1 1863.1 ],"[0.2548125633232016, 0.27254305977710236, 0.23...",[15]
1778,HMDB0000667,m05,3.35,1,"['8.28', '4.48']",['16'],dd,3.18,3.53,"[3.34, 3.35, 3.36, 3.37]",[2007.34 2013.35 2019.36 2025.37],"[0.25606095589933037, 0.26368044331563145, 0.2...",[16]
2993,HMDB0000667,m04,6.77,2,[''],"['5', '3']",d,6.74,6.78,"[6.76, 6.77, 6.77, 6.78]",[4062.76 4068.77 4068.77 4074.78],"[0.28920011550678604, 0.12128212532486285, 0.1...","[5, 3]"
3003,HMDB0000667,m03,6.80,2,['8.58'],"['13', '9']",d,6.78,6.82,"[6.78, 6.79, 6.81]",[4074.78 4080.79 4092.81],"[0.40782122905027934, 0.28982150156697095, 0.3...","[13, 9]"
3026,HMDB0000667,m02,6.84,2,[''],"['6', '2']",d,6.82,6.87,"[6.83, 6.84, 6.84, 6.85]",[4104.83 4110.84 4110.84 4116.85],"[0.43540306664642475, 0.14665249734325186, 0.1...","[6, 2]"
3226,HMDB0000667,m01,7.20,2,['8.58'],"['12', '10']",d,7.17,7.23,"[-0.009679579148732458, 0.010320420851267116]",[4321.19 4333.21],"[0.5160210425633668, 0.48397895743663316]","[12, 10]"


In [21]:
# Looking for Ibupeofen
query = [{'range': (7.07, 7.2), 'mult': 'd', 'ppm': [7.1228, 7.136], 'heights': [0.83,1]},
        {'range': (7.2, 7.35), 'mult': 'd', 'ppm': [7.241, 7.2542], 'heights': [1, 0.83]}]
     #   {'range': (0.88, 0.96), 'mult': 'd', 'ppm': [0.916, 0.927], 'heights': [1.0,1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
23,HMDB0004811,"2,4-Dichlorophenol",2.374171
17,HMDB0001925,Ibuprofen,1.923175
4,HMDB0000667,L-Thyronine,1.619625
16,HMDB0001885,3-Chlorotyrosine,1.408662
15,HMDB0001868,5-Methoxysalicylic acid,1.323297
14,HMDB0001713,m-Coumaric acid,1.222001


In [22]:
nmrdb[nmrdb['accession'] == 'HMDB0001925']
#[HMDB0001872, HMDB01872, HMDB01925]

,accession,multiplet,shift1(ppm),hs,j(hz),atom1,type,from,to,ppm,hz,heights,assigned atoms
141,HMDB0001925,m07,0.90,6,['6.61'],"['9', '12']",d,0.86,0.94,"[0.89, 0.91]","[446.4, 453.0]","[0.48971781395111497, 0.5102821860488851]","[9, 12]"
562,HMDB0001925,m06,1.51,3,['7.16'],['11'],d,1.48,1.54,"[1.5, 1.52]","[750.1, 757.3]","[0.4859700840703134, 0.5140299159296867]",[11]
810,HMDB0001925,m05,1.85,1,[''],['8'],m,1.79,1.91,"[1.81, 1.82, 1.83, 1.85, 1.86, 1.88, 1.89]","[903.3, 910.3, 917.0, 923.7, 930.4, 937.3, 944.0]","[0.03258265452803066, 0.10828941063727837, 0.2...",[8]
1323,HMDB0001925,m04,2.45,2,['7.18'],['7'],d,2.42,2.48,"[2.44, 2.46]","[1221.1, 1228.3]","[0.5146812080536913, 0.4853187919463087]",[7]
2039,HMDB0001925,m03,3.72,1,['7.17'],['10'],q,3.67,3.77,"[3.7, 3.71, 3.73, 3.74]","[1849.6, 1856.8, 1863.9, 1871.1]","[0.10770913067249863, 0.37233460907599775, 0.3...",[10]
3176,HMDB0001925,m02,7.11,2,['8.10'],"['2', '6']",d,7.08,7.13,"[-0.005830028328612292, 0.004169971671388383]","[3548.0, 3556.1]","[0.4169971671388102, 0.5830028328611898]","[2, 6]"
3237,HMDB0001925,m01,7.23,2,['8.03'],"['3', '5']",d,7.20,7.25,"[-0.004422520389371517, 0.005577479610629158]","[3608.0, 3616.0]","[0.5577479610628782, 0.44225203893712184]","[3, 5]"


In [23]:
# Looking for Ethyl vinyl ether
query = [{'range': (3.72, 3.82), 'mult': 'q', 'ppm': [3.7440, 3.7620, 3.7799, 3.7969], 'heights': [1.0, 3.0, 3.0, 1.0]},
         {'range': (1.25, 1.36), 'mult': 'd', 'ppm': [1.2903, 1.3081, 1.3256]	, 'heights': [1.0, 2.0, 1.0]}]

result = multiple_query(query, nmrdb, df)
result.head(6) 

,accession,name,similarity
0,HMDB0000174,L-Fucose,0.792581
6,HMDB0001645,L-Norleucine,0.500000
2,HMDB0000832,Capryloylglycine,0.400000
4,HMDB0000953,Suberylglycine,0.400000
1,HMDB0000651,Decanoylcarnitine,0.222222
3,HMDB0000848,Stearoylcarnitine,0.222222
